# Encoded Chain Extract Document Compressor

The `LLMEncodedChainExtractor` is an optimized version of the `LLMChainExtractor` which uses an encoding technique to fetch only the indices of the relevant sequence instead of having the LLM regenerate the sequence verbatim.

This results in *~2X* faster compression and completion tokens which are orders of magnitude less.

The code snippet below performs this comparison on a sample document.

In [4]:
from time import time

from langchain.callbacks import get_openai_callback
from langchain.llms import OpenAI
from langchain.retrievers.document_compressors import (
    LLMChainExtractor,
    LLMEncodedChainExtractor,
)
from langchain.schema import Document

In [5]:
# Setup
chain_extract_compressor = LLMChainExtractor.from_llm(OpenAI())
encoded_chain_extract_compressor = LLMEncodedChainExtractor.from_llm(OpenAI(), pipeline="sentencizer")
texts = [
    "The Roman Empire followed the Roman Republic.",
    "I love chocolate chip cookies—my mother makes great cookies.",
    "The first Roman emperor was Caesar Augustus.",
    "Don't you just love Caesar salad?",
    "The Roman Empire collapsed in 476 AD after the fall of Rome.",
    "Let's go to Olive Garden!",
]

doc = Document(page_content=" ".join(texts))

In [ ]:
# Benchmark LLMChainExtractor
chain_extract_total_time = 0
chain_extract_total_tokens = 0
for _ in range(10):
    with get_openai_callback() as cb:
        start = time()
        chain_extract_compressor.compress_documents(
            [doc], "Tell me about the Roman Empire"
        )
        chain_extract_total_time += time() - start
        chain_extract_total_tokens += cb.completion_tokens

In [7]:
# Benchmark LLMEncodedChainExtractor
encoded_chain_extract_total_time = 0
encoded_chain_extract_total_tokens = 0
for _ in range(10):
    with get_openai_callback() as cb:
        start = time()
        encoded_chain_extract_compressor.compress_documents(
            [doc], "Tell me about the Roman Empire"
        )
        encoded_chain_extract_total_time += time() - start
        encoded_chain_extract_total_tokens += cb.completion_tokens

In [8]:
# Print results
print(
    f"LLMChainExtractor: {chain_extract_total_time/10} seconds,\
          {chain_extract_total_tokens/10} tokens"
)
print(
    f"LLMEncodedChainExtractor: {encoded_chain_extract_total_time/10} seconds,\
          {encoded_chain_extract_total_tokens/10} tokens"
)

LLMChainExtractor: 1.0736155033111572 seconds,          30.3 tokens
LLMEncodedChainExtractor: 0.48770740032196047 seconds,          5.0 tokens
